# Concurrency

A **concurrent** program is one consisting of multiple threads executing "at the same time" and independently.

A **thread** is an independent line of execution within a process.

A thread has its own set of registers (its context!) on the CPU.

A thread has its own stack.

A thread its own instruction pointer (aka program counter).

Within a process, the rest of the address space is shared amongst all of its threads:

- heap
- code block
- static global block


<br>
<img src="images/01-AS.png" width="500">
<br>

## Why talk about concurrency in OS?

The OS is the first concurrent program and needs to support concurrency in order to exist.

The mechanisms to support concurrency are critical to a computer operating safely and must be with the OS's purview.

Concurrency is useful if we can paralellize programs. If we can split some algorithm or overal program into threads and execute them independently at the same time, we can save time overall.

Concurrency is also very useful in IO intensive applications.

Web servers are multithreaded. When a request comes in, a new thread is created specifically to handle that request so that the server can continue to accept other requests and handle multiple requests at once.

## The Major Challenge of Concurrency

Concurrent programs can execute in a non-deterministic order because we (the programmers) don't have control over when the threads will be executed.

The order is determined by the CPU scheduler.

The is especially problematic when multiple threads are attempting to access shared data.

If multiple threads are trying to update shared data, and they can be interrupted while doing so, that data can get corrupted.


## Terminology

Any set of instructions which should not be interrupted if want our program to be correct is known as **critical section**.

We want all critical sections to be **atomic**.

A set of instructions is **atomic** if it will run to completion for any individual thread before any other thread can start executing them.

If multiple threads end up in the same critical section at the same time, we have a **race condition**.

To avoid race conditions, to ensure that critical sections are atomic, we use **locks**.

We will explore how to use and build locks.

# How to use a lock

A **lock** is a variable that only one thread can hold at a time.

If we put a lock around a critical section, only one thread can access it at a time. We are making that critical section mutually exclusive.

To use a lock, a program instaniates it, then attempts to hold it before entering some critical section, and finally upon leaving the critical section, releases it.

```c
lock_t mutex; //declare the lock, mutex for mutual exclusion
init(&mutex); // initialize the lock
// ..
lock(&mutex); // attemp to grab the lock
sharedVariable++; // critical section
unlock(&mutex); // release the lock
```

If thread A holds the lock when thread attempts to grab it, then thread B must wait until A releases the lock before it can get it and continue execution.

In this way, we ensure that only one thread can be in the critical section at a time.

A program can create and use multiple locks.

If there are multiple critical sections, it is best to have a lock per critical section so that different threads can be in different critical sections at the same time.

# Building a Lock

## Goals

**Correctness**: Our lock should correctly ensure mutually exclusive access to critical sections

**Fairness**: If we multiple threads all vying for the same lock, eventually each thread should get the lock. We don't want to starve our threads.

**Performance**: Minimize overhead. Within a single thread, the overhead should be minimal. Over the whole system, we want to minimize the overall time spent waiting for locks.